In [250]:
import json
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from itertools import chain
from collections import Counter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

### Read data

In [251]:
# Load the pickled list
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/zeroshot/results/comics35_zs_pg_llama-3-8b-Instruct-bnb-4bit/results.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [252]:
results

{'grounds': ["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
  "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
  "[['joy'], ['joy'], ['anger'], ['anger']]",
  "[['fear', 'surprise']]",
  "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
  "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
  "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
  "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
  "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sad

In [253]:
#all_labels = ["Anger", "Disgust", "Fear", "Sadness", "Surprise", "Joy"]

In [254]:
grounds = results['grounds']
predictions = results['predictions']

In [255]:
len(grounds), len(predictions)

(874, 874)

In [256]:
grounds

["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
 "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['joy'], ['joy'], ['anger'], ['anger']]",
 "[['fear', 'surprise']]",
 "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
 "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
 "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
 "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
 "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sadness'], ['anger']]",

In [257]:
predictions

['{\n"page_utterance_emotions": [\n    ["anger", "disgust", "fear"],\n    ["sadness", "disgust", "fear"],\n    ["joy"],\n    ["fear", "sadness"],\n    ["anger"],\n    ["sadness"],\n    ["anger"],\n    ["surprise"],\n    ["surprise", "joy"],\n    ["fear", "sadness"]\n]\n}',
 'Here is the JSON output:\n\n{\n"page_utterance_emotions": [\n    ["anger"],\n    ["surprise", "disgust"],\n    ["fear"],\n    ["surprise", "disgust", "anger"],\n    ["joy", "disgust"],\n    ["surprise", "disgust"],\n    ["neutral"],\n    ["anger"],\n    ["anger", "fear"]\n]\n}',
 '{\n"page_utterance_emotions": [\n  ["anger"],\n  ["surprise"],\n  ["fear", "disgust"],\n  ["joy", "anger"]\n]\n}',
 '{"page_utterance_emotions": [["anger"]]}',
 'Here is the JSON output:\n\n{\n"page_utterance_emotions": [\n[\n["joy", "anger"]\n],\n[\n["joy", "anger"]\n],\n[\n["surprise", "fear"]\n],\n[\n["disgust", "anger"]\n],\n[\n["surprise", "joy"]\n]\n]\n}',
 'Here is the JSON output:\n\n{\n"page_utterance_emotions": [\n    ["surprise

In [258]:
#grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]

In [259]:
import re

In [260]:
processed_preds = []
bad_idx = []

for i, input_string in enumerate(predictions):
    
    json_content = re.search(r'\{.*\}', input_string, re.DOTALL)
    
    if json_content:
        json_string = json_content.group()
        processed_preds.append(json_string)
    else:    
        bad_idx.append(i)
        print(i)
        

28
29
39
43
50
66
70
74
88
102
112
113
124
129
130
136
139
152
154
162
168
178
179
187
197
198
200
211
220
243
252
253
256
273
274
281
283
286
289
297
301
305
314
316
326
328
331
336
339
341
354
358
369
390
423
429
438
447
460
473
480
486
489
492
496
497
502
521
538
542
545
552
563
566
572
579
580
581
585
586
588
591
594
604
605
607
610
615
621
623
625
627
636
643
669
671
677
687
711
714
718
727
728
745
758
761
768
769
773
774
783
788
800
803
804
820
825
832
835
850
864
872


In [261]:
predictions[29]

'{\n"page_utterance_emotions": [\n    ["joy"],\n    ["gratitude"],\n    ["neutral"]\n],\n[\n    ["surprise", "fear"]\n],\n[\n    ["fear", "anger"]\n]\n]'

In [262]:
# predictions = [json.loads(x)["page_utterance_emotions"] for x in predictions]

In [263]:
len(grounds), len(processed_preds)

(874, 752)

In [264]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    #del predictions[idx]

In [265]:
len(grounds), len(processed_preds)

(752, 752)

In [266]:
predictions = [json.loads(x)["page_utterance_emotions"] for x in processed_preds]

In [267]:
len(grounds), len(predictions)

(752, 752)

In [268]:
grounds

["[['anger'], ['anger'], ['fear'], ['fear'], ['fear', 'sadness'], ['sadness'], ['anger'], ['surprise'], ['surprise'], ['fear', 'surprise']]",
 "[['fear'], ['anger'], ['surprise'], ['anger'], ['joy'], ['anger'], ['anger'], ['anger'], ['anger']]",
 "[['joy'], ['joy'], ['anger'], ['anger']]",
 "[['fear', 'surprise']]",
 "[['anger'], ['anger'], ['fear'], ['fear', 'surprise'], ['anger', 'joy'], ['anger']]",
 "[['sadness'], ['fear'], ['joy'], ['neutral'], ['surprise'], ['fear'], ['surprise'], ['joy'], ['surprise'], ['sadness', 'surprise']]",
 "[['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']]",
 "[['fear'], ['fear'], ['anger'], ['fear'], ['anger', 'surprise'], ['fear'], ['fear'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['anger'], ['anger']]",
 "[['fear'], ['fear'], ['anger'], ['anger', 'fear', 'surprise'], ['fear', 'sadness'], ['fear', 'sadness'], ['anger'], ['anger'], ['surprise'], ['fear'], ['anger', 'surprise'], ['anger'], ['anger', 'sadness'], ['anger']]",

In [269]:
import ast

In [270]:
grounds = [ast.literal_eval(x) for x in grounds]

In [271]:
grounds

[[['anger'],
  ['anger'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['sadness'],
  ['anger'],
  ['surprise'],
  ['surprise'],
  ['fear', 'surprise']],
 [['fear'],
  ['anger'],
  ['surprise'],
  ['anger'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger']],
 [['joy'], ['joy'], ['anger'], ['anger']],
 [['fear', 'surprise']],
 [['anger'],
  ['anger'],
  ['fear'],
  ['fear', 'surprise'],
  ['anger', 'joy'],
  ['anger']],
 [['sadness'],
  ['fear'],
  ['joy'],
  ['neutral'],
  ['surprise'],
  ['fear'],
  ['surprise'],
  ['joy'],
  ['surprise'],
  ['sadness', 'surprise']],
 [['surprise'], ['fear', 'sadness'], ['surprise'], ['surprise'], ['joy']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['fear'],
  ['anger', 'surprise'],
  ['fear'],
  ['fear'],
  ['fear', 'sadness'],
  ['surprise'],
  ['surprise'],
  ['anger'],
  ['anger']],
 [['fear'],
  ['fear'],
  ['anger'],
  ['anger', 'fear', 'surprise'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['anger'],
  ['anger'],
  ['surpris

In [272]:
predictions

[[['anger', 'disgust', 'fear'],
  ['sadness', 'disgust', 'fear'],
  ['joy'],
  ['fear', 'sadness'],
  ['anger'],
  ['sadness'],
  ['anger'],
  ['surprise'],
  ['surprise', 'joy'],
  ['fear', 'sadness']],
 [['anger'],
  ['surprise', 'disgust'],
  ['fear'],
  ['surprise', 'disgust', 'anger'],
  ['joy', 'disgust'],
  ['surprise', 'disgust'],
  ['neutral'],
  ['anger'],
  ['anger', 'fear']],
 [['anger'], ['surprise'], ['fear', 'disgust'], ['joy', 'anger']],
 [['anger']],
 [[['joy', 'anger']],
  [['joy', 'anger']],
  [['surprise', 'fear']],
  [['disgust', 'anger']],
  [['surprise', 'joy']]],
 [['surprise'],
  ['fear'],
  ['anger'],
  [],
  ['joy'],
  ['fear'],
  ['fear'],
  ['fear'],
  ['neutral'],
  ['neutral', 'joy']],
 [[['surprise']], [['disgust']], [['fear']], [['joy']], [['joy', 'neutral']]],
 [[['fear', 'surprise']],
  [['neutral']],
  [['anger', 'surprise']],
  [['neutral']],
  [['anger', 'fear']],
  [['fear', 'neutral']],
  [['fear', 'neutral']],
  [['anger', 'fear']],
  [['neutral

In [273]:
bad_idx = []

for idx, (i,j) in enumerate(zip(grounds, predictions)):
    if len(i) != len(j):
        print(idx, len(i), len(j))
        bad_idx.append(idx)

4 6 5
7 12 13
14 11 12
23 12 11
26 14 12
33 10 9
35 8 7
39 13 12
42 11 10
54 6 5
55 8 7
63 3 2
64 10 9
65 13 11
67 18 19
68 16 20
69 16 15
74 13 15
77 9 10
81 10 11
85 8 7
90 11 12
92 16 15
94 20 27
95 15 17
97 12 14
102 23 24
105 13 12
106 14 15
107 10 9
109 11 10
111 13 12
112 13 11
113 17 16
114 4 3
115 12 13
116 12 11
121 12 10
124 9 8
129 3 4
130 9 8
131 12 11
132 4 3
135 13 12
137 7 6
141 13 12
142 18 14
147 13 12
151 11 10
152 14 17
158 10 13
160 17 18
168 13 12
170 4 3
178 22 21
184 4 3
188 15 19
192 7 6
197 11 14
201 15 14
203 17 15
206 9 8
211 10 8
212 11 10
213 12 11
218 15 16
219 17 14
223 14 13
229 9 10
239 19 15
240 13 15
241 15 14
243 15 16
249 11 10
250 15 17
252 17 14
253 15 13
260 7 6
261 11 10
264 13 14
267 11 12
269 12 13
270 16 15
273 15 11
274 13 12
275 12 11
279 14 12
281 14 12
285 4 5
288 1 2
292 7 6
293 12 11
296 9 8
297 11 12
305 9 10
308 16 15
310 12 10
311 12 13
313 11 12
315 10 11
316 15 16
319 10 9
327 8 9
328 11 10
331 8 9
338 7 6
347 7 6
349 14 16
350 12

In [274]:
len(bad_idx)

207

In [275]:
bad_idx.sort(reverse=True)

# Remove elements from 'grounds' at the specified indices
for idx in bad_idx:
    
    del grounds[idx]
    del predictions[idx]

In [276]:
len(grounds), len(predictions)

(545, 545)

In [277]:
grounds = [item for sublist in grounds for item in sublist]
predictions = [item for sublist in predictions for item in sublist]

In [278]:
#grounds = [[item for inner in outer for item in inner] for outer in grounds]
#predictions = [[item for inner in outer for item in inner] for outer in predictions]

In [279]:
len(grounds), len(predictions)

(3695, 3695)

In [280]:
grounds

[['anger'],
 ['anger'],
 ['fear'],
 ['fear'],
 ['fear', 'sadness'],
 ['sadness'],
 ['anger'],
 ['surprise'],
 ['surprise'],
 ['fear', 'surprise'],
 ['fear'],
 ['anger'],
 ['surprise'],
 ['anger'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['anger'],
 ['fear', 'surprise'],
 ['sadness'],
 ['fear'],
 ['joy'],
 ['neutral'],
 ['surprise'],
 ['fear'],
 ['surprise'],
 ['joy'],
 ['surprise'],
 ['sadness', 'surprise'],
 ['surprise'],
 ['fear', 'sadness'],
 ['surprise'],
 ['surprise'],
 ['joy'],
 ['fear'],
 ['fear'],
 ['anger'],
 ['anger', 'fear', 'surprise'],
 ['fear', 'sadness'],
 ['fear', 'sadness'],
 ['anger'],
 ['anger'],
 ['surprise'],
 ['fear'],
 ['anger', 'surprise'],
 ['anger'],
 ['anger', 'sadness'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['fear'],
 ['joy'],
 ['fear', 'sadness'],
 ['anger', 'disgust', 'surprise'],
 ['anger', 'disgust'],
 ['fear'],
 ['anger', 'disgust'],
 ['fear', 'surprise'],
 ['anger'],
 ['joy'],
 ['joy'],
 ['anger'],
 ['fear

In [281]:
predictions

[['anger', 'disgust', 'fear'],
 ['sadness', 'disgust', 'fear'],
 ['joy'],
 ['fear', 'sadness'],
 ['anger'],
 ['sadness'],
 ['anger'],
 ['surprise'],
 ['surprise', 'joy'],
 ['fear', 'sadness'],
 ['anger'],
 ['surprise', 'disgust'],
 ['fear'],
 ['surprise', 'disgust', 'anger'],
 ['joy', 'disgust'],
 ['surprise', 'disgust'],
 ['neutral'],
 ['anger'],
 ['anger', 'fear'],
 ['anger'],
 ['surprise'],
 ['fear', 'disgust'],
 ['joy', 'anger'],
 ['anger'],
 ['surprise'],
 ['fear'],
 ['anger'],
 [],
 ['joy'],
 ['fear'],
 ['fear'],
 ['fear'],
 ['neutral'],
 ['neutral', 'joy'],
 [['surprise']],
 [['disgust']],
 [['fear']],
 [['joy']],
 [['joy', 'neutral']],
 [['fear', 'surprise']],
 [['disgust', 'fear']],
 [['anger', 'surprise']],
 [['sadness', 'anger']],
 [['fear']],
 [['fear', 'surprise']],
 [['anger', 'surprise']],
 [['surprise']],
 [['fear', 'sadness']],
 [['anger', 'fear']],
 [['surprise', 'fear']],
 [['fear', 'sadness']],
 [['anger', 'fear']],
 [['anger', 'fear']],
 ['joy'],
 ['neutral'],
 ['f

In [287]:
predictions = [
    sublist[0] if isinstance(sublist, list) and len(sublist) == 1 and isinstance(sublist[0], list) else sublist
    for sublist in predictions
]

In [291]:
predictions

[['anger', 'disgust', 'fear'],
 ['sadness', 'disgust', 'fear'],
 ['joy'],
 ['fear', 'sadness'],
 ['anger'],
 ['sadness'],
 ['anger'],
 ['surprise'],
 ['surprise', 'joy'],
 ['fear', 'sadness'],
 ['anger'],
 ['surprise', 'disgust'],
 ['fear'],
 ['surprise', 'disgust', 'anger'],
 ['joy', 'disgust'],
 ['surprise', 'disgust'],
 ['neutral'],
 ['anger'],
 ['anger', 'fear'],
 ['anger'],
 ['surprise'],
 ['fear', 'disgust'],
 ['joy', 'anger'],
 ['anger'],
 ['surprise'],
 ['fear'],
 ['anger'],
 [],
 ['joy'],
 ['fear'],
 ['fear'],
 ['fear'],
 ['neutral'],
 ['neutral', 'joy'],
 ['surprise'],
 ['disgust'],
 ['fear'],
 ['joy'],
 ['joy', 'neutral'],
 ['fear', 'surprise'],
 ['disgust', 'fear'],
 ['anger', 'surprise'],
 ['sadness', 'anger'],
 ['fear'],
 ['fear', 'surprise'],
 ['anger', 'surprise'],
 ['surprise'],
 ['fear', 'sadness'],
 ['anger', 'fear'],
 ['surprise', 'fear'],
 ['fear', 'sadness'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['joy'],
 ['neutral'],
 ['fear', 'sadness'],
 ['neutral', 'joy'],

In [292]:
predictions = [
    [item for sublist in lst for item in sublist] if all(isinstance(i, list) for i in lst) else lst
    for lst in predictions
]

In [293]:
predictions

[['anger', 'disgust', 'fear'],
 ['sadness', 'disgust', 'fear'],
 ['joy'],
 ['fear', 'sadness'],
 ['anger'],
 ['sadness'],
 ['anger'],
 ['surprise'],
 ['surprise', 'joy'],
 ['fear', 'sadness'],
 ['anger'],
 ['surprise', 'disgust'],
 ['fear'],
 ['surprise', 'disgust', 'anger'],
 ['joy', 'disgust'],
 ['surprise', 'disgust'],
 ['neutral'],
 ['anger'],
 ['anger', 'fear'],
 ['anger'],
 ['surprise'],
 ['fear', 'disgust'],
 ['joy', 'anger'],
 ['anger'],
 ['surprise'],
 ['fear'],
 ['anger'],
 [],
 ['joy'],
 ['fear'],
 ['fear'],
 ['fear'],
 ['neutral'],
 ['neutral', 'joy'],
 ['surprise'],
 ['disgust'],
 ['fear'],
 ['joy'],
 ['joy', 'neutral'],
 ['fear', 'surprise'],
 ['disgust', 'fear'],
 ['anger', 'surprise'],
 ['sadness', 'anger'],
 ['fear'],
 ['fear', 'surprise'],
 ['anger', 'surprise'],
 ['surprise'],
 ['fear', 'sadness'],
 ['anger', 'fear'],
 ['surprise', 'fear'],
 ['fear', 'sadness'],
 ['anger', 'fear'],
 ['anger', 'fear'],
 ['joy'],
 ['neutral'],
 ['fear', 'sadness'],
 ['neutral', 'joy'],

In [282]:
for idx, (i,j) in enumerate(zip(grounds, predictions)):
    if len(i) != len(j):
        print(idx, len(i), len(j))

0 1 3
1 1 3
3 1 2
4 2 1
8 1 2
11 1 2
13 1 3
14 1 2
15 1 2
18 1 2
21 1 2
22 1 2
23 2 1
27 1 0
35 2 1
42 3 1
43 2 1
44 2 1
49 2 1
51 2 1
55 1 2
56 1 2
57 2 3
58 3 2
60 1 3
63 1 2
69 1 2
70 1 2
72 1 2
73 1 2
74 1 3
75 1 2
81 2 1
85 2 1
86 2 1
87 2 1
89 2 1
91 1 2
93 2 1
98 1 2
99 2 1
100 1 2
101 1 2
104 1 2
105 2 1
107 1 2
109 1 2
111 1 2
112 1 2
116 2 1
118 1 2
122 2 1
125 1 2
129 2 1
133 1 2
134 2 1
137 2 1
138 1 2
139 1 2
146 2 1
155 2 1
156 1 2
164 1 3
165 1 3
167 1 2
168 1 2
169 2 3
172 1 2
173 2 1
174 3 1
175 2 1
182 1 2
183 1 2
188 2 1
189 2 1
190 2 1
191 2 1
197 2 1
204 2 1
211 1 2
212 1 2
213 1 2
215 1 2
216 1 2
217 1 2
220 2 1
221 1 2
225 1 2
229 2 1
231 2 1
232 2 1
233 2 1
235 2 1
241 3 2
243 2 1
245 1 2
249 2 1
250 2 1
251 2 1
259 1 2
260 2 3
261 2 3
262 1 3
264 1 2
265 1 2
267 2 1
276 2 1
277 1 2
280 2 1
281 2 1
289 1 2
290 2 1
294 2 1
296 2 1
297 3 2
298 3 2
302 2 1
303 2 1
310 2 1
311 2 1
312 2 1
315 2 1
317 2 1
318 2 1
319 2 1
320 3 1
321 2 1
322 1 2
325 1 2
329 1 2
332 1 

In [283]:
# grounds = [item for sublist1 in grounds for sublist2 in sublist1 for item in sublist2]
# predictions = [item for sublist1 in predictions for sublist2 in sublist1 for item in sublist2]

In [294]:
len(grounds), len(predictions)

(3695, 3695)

In [295]:
mlb = MultiLabelBinarizer()

In [296]:
y_true_mhot = mlb.fit_transform(grounds)
y_pred_mhot = mlb.transform(predictions)

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['admiration', 'aggression', 'amusement', 'anticipation', 'apologetic', 'apology', 'appeal', 'appreciation', 'assertiveness', 'awkwardness', 'caution', 'challenge', 'coldness', 'concern', 'confidence', 'confusion', 'curiosity', 'defeat', 'determination', 'disappointment', 'distrust', 'embarrassment', 'empathy', 'encouragement', 'excitement', 'frustration', 'gratitude', 'greed', 'guilt', 'happiness', 'hatred', 'helpfulness', 'hope', 'irony', 'longing', 'love', 'motivation', 'negotiation', 'optimism', 'pain', 'pleading', 'power', 'pride', 'regret', 'relief', 'reluctance', 'sarcasm', 'skepticism', 'suspicion', 'trust', 'urgency'] will be ignored
  warnings.warn(


In [297]:
y_true_mhot.shape

(3695, 7)

In [298]:
y_true_mhot.shape, y_pred_mhot.shape

((3695, 7), (3695, 7))

In [299]:
class_labels = mlb.classes_

In [300]:
class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [301]:
print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.596     0.412     0.487      1321
     disgust      0.134     0.432     0.204       162
        fear      0.362     0.375     0.369       933
         joy      0.424     0.357     0.387       838
     neutral      0.098     0.623     0.170       228
     sadness      0.512     0.273     0.356       884
    surprise      0.409     0.353     0.379       972

   micro avg      0.339     0.373     0.355      5338
   macro avg      0.362     0.403     0.336      5338
weighted avg      0.445     0.373     0.387      5338
 samples avg      0.330     0.380     0.334      5338



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [313]:
with open("/Utilisateurs/umushtaq/emotion_analysis_comics/finetuning/saved_models/comics35_pg_old_Meta-Llama-3.1-70B-Instruct-bnb-4bit/classification_report_3.pickle", 'wb') as fh:
    
    pickle.dump(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3, output_dict=True), fh)

In [ ]:
# grounds = results['grounds']
# predictions = results['predictions']

In [ ]:
# grounds = [json.loads(x)["page_utterance_emotions"] for x in grounds]
# predictions = [json.loads(x)["page_utterance_emotions"] for x in predictions]
#predictions = [x['page_utterance_emotions'] for x in predictions]

In [ ]:
#grounds

[[['surprise', 'joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['surprise'],
  ['joy'],
  ['joy'],
  ['neutral'],
  ['neutral']],
 [['neutral'],
  ['neutral'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['neutral'],
  ['sadness'],
  ['sadness']],
 [['anger', 'sadness'],
  ['anger', 'sadness'],
  ['anger', 'sadness'],
  ['fear', 'surprise'],
  ['surprise'],
  ['joy'],
  ['anger', 'surprise'],
  ['joy'],
  ['joy'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['surprise', 'joy'],
  ['fear', 'sadness'],
  ['fear', 'sadness'],
  ['fear', 'surprise']],
 [['anger', 'disgust'],
  ['anger', 'disgust'],
  ['anger', 'disgust'],
  ['fear', 'sadness'],
  ['fear', 'sadness', 'surprise'],
  ['sadness'],
  ['sadness'],
  ['fear', 'sadness'],
  ['sadness', 'surprise'],
  ['sadness', 'surprise'],
  ['joy'],
  ['anger'],
  ['anger'],
  ['anger'],
  ['anger', 'disgust'],
  ['joy'],
  ['joy'],
  ['surprise', 'joy'],
  ['surprise', 'joy'],
  ['anger', 'surprise'],
  ['anger', 

In [ ]:
#grounds_u = [[item for sublist in outer_list for item in sublist] for outer_list in grounds]

In [ ]:
#predictions_u = [[item for sublist in outer_list for item in sublist] for outer_list in predictions]

In [ ]:
#en(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
#grounds_u

[['surprise',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'joy',
  'joy',
  'surprise',
  'joy',
  'joy',
  'neutral',
  'neutral'],
 ['neutral',
  'neutral',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'neutral',
  'sadness',
  'sadness'],
 ['anger',
  'sadness',
  'anger',
  'sadness',
  'anger',
  'sadness',
  'fear',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'joy',
  'joy',
  'joy',
  'anger',
  'anger',
  'surprise',
  'joy',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'fear',
  'surprise'],
 ['anger',
  'disgust',
  'anger',
  'disgust',
  'anger',
  'disgust',
  'fear',
  'sadness',
  'fear',
  'sadness',
  'surprise',
  'sadness',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'surprise',
  'sadness',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'disgust',
  'joy',
  'joy',
  'surprise',
  'joy',
  'surprise',
  'joy',
  'anger',
  'surprise',
  'anger',
  'surprise'],
 ['neutral', 'joy', 'joy'],
 ['neutral',
 

In [ ]:
#predictions_u

[['joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy'],
 ['neutral',
  'neutral',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'neutral',
  'anger',
  'disgust',
  'surprise',
  'anger',
  'disgust',
  'surprise'],
 ['anger',
  'anger',
  'sadness',
  'anger',
  'anger',
  'surprise',
  'anger',
  'surprise',
  'sadness',
  'anger',
  'surprise',
  'anger',
  'joy',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'surprise',
  'joy',
  'joy'],
 ['anger',
  'anger',
  'anger',
  'sadness',
  'sadness',
  'sadness',
  'sadness',
  'anger',
  'surprise',
  'surprise',
  'joy',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'anger',
  'joy',
  'anger',
  'anger'],
 ['joy', 'joy', 'joy'],
 ['sadness',
  'fear',
  'surprise',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'sadness',
  'joy',
  'sadness',
  'fear',
  'sadness',
  'joy',
  'sadness',
  'joy',
  'joy',
  'sadness',
  'joy',
  'anger',

In [ ]:
#len(grounds_u), len(predictions_u)

(156, 156)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()

In [ ]:
#y_true_mhot = mlb.fit_transform(grounds_u)

In [ ]:
#y_true_mhot.shape

(156, 7)

In [ ]:
#y_true_mhot

array([[0, 0, 0, ..., 1, 0, 1],
       [1, 1, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 0, 1, 1],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#y_pred_mhot = mlb.transform(predictions_u)

In [ ]:
#y_pred_mhot.shape

(156, 7)

In [ ]:
#y_pred_mhot

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 1, 0, 1],
       [1, 0, 0, ..., 0, 1, 1],
       ...,
       [1, 0, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 0, 0]])

In [ ]:
#class_labels = mlb.classes_

In [ ]:
#class_labels

array(['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness',
       'surprise'], dtype=object)

In [ ]:
#print(classification_report(y_true_mhot, y_pred_mhot, target_names=class_labels, digits=3))

              precision    recall  f1-score   support

       anger      0.929     0.778     0.847       117
     disgust      0.318     0.250     0.280        28
        fear      0.756     0.626     0.685        99
         joy      0.855     0.678     0.756        87
     neutral      0.778     0.311     0.444        45
     sadness      0.750     0.734     0.742        94
    surprise      0.948     0.879     0.912       124

   micro avg      0.829     0.692     0.754       594
   macro avg      0.762     0.608     0.667       594
weighted avg      0.825     0.692     0.746       594
 samples avg      0.835     0.692     0.727       594

